In [ ]:

#####################################
################ LDA ################
#####################################


from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string


#### import data###

kdesc= pd.read_excel('kaggle_desc.xlsx')
kd = kdesc[['Competition Id','Overview']].set_index('Competition Id')

#doc_complete = kd[['Overview']]
kdesc.to_pickle('kagdesc.pkl')
kdesc = pd.read_pickle('kagdesc.pkl')

kd.to_csv('kaggdescr.csv')

### preprocess and vectorize data####
### gensim tutorial

docs = kdesc[['Overview']]

from nltk.tokenize import RegexpTokenizer

## initialize pattern
tokenizer = RegexpTokenizer(r'\w+')
## convert to lower
docs['Overview'] = docs['Overview'].str.lower()
## convert to list
docs = docs['Overview'].tolist()
## tokenize##
for idx in range(len(docs)):
    docs[idx]= tokenizer.tokenize(docs[idx])

## remove numbers, but not words that contain numbers
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

## remove words that are only one character
docs = [[token for token in doc if len(token)>1] for doc in docs]

## lemmatize words
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

## remove stopwords
stop = set(stopwords.words('english'))
docs = [[token for token in doc if token not in stop] for doc in docs]

## compute bigrams
from gensim.models import Phrases

## add bigrams and trigrams to docs (only those that appear 3 times or more).
bigram = Phrases(docs, min_count = 10) # ran three times with min count 3, 5, 10
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            #Token is a bigram, add to document
            docs[idx].append(token)

pd.DataFrame(docs).to_pickle('kaggle_topicsraw.pkl')

## remove rare and common tokens
from gensim.corpora import Dictionary

## create a dictionary representation of the docs
dictionary = Dictionary(docs)

### rough work

dictionary.filter_extremes


####


## filter out words that occur in less than 1 documents or more than 40% of the documents
dictionary.filter_extremes(no_below = 1,no_above = 0.4)

#Vectorize data
## Bag of words representation of the documents
corpus = [dictionary.doc2bow(doc) for doc in docs]

## Bag of words to TfIdf 


print('no. of unique tokens: %d' % len(dictionary) )
print('no of documents: %s' % len(corpus))


### train the LDA model###
from gensim.models import LdaModel

## set training parameters

num_topics = 10
chunksize = 300
passes = 30
iterations = 500
eval_every = None 

## Make a index to word dictionary

temp = dictionary[0]
id2word = dictionary.id2token

%time model = LdaModel(corpus = corpus, id2word = id2word, chunksize = chunksize, \
                       alpha = 'auto', eta ='auto', iterations = iterations,\
                       num_topics = num_topics,passes = passes, eval_every = eval_every)


top_topics = model.top_topics(corpus, topn = 20)

#Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics
 
avg_topic_coherence = sum([t[1] for t in top_topics])/ num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence )

from pprint import pprint
pprint(top_topics)






##########################################################
##########################################################
##########################################################


### import dataset###

kdesc= pd.read_excel('kaggle_desc.xlsx')
kdesc.head()

kdesc.to_pickle('kagdesc.pkl')
kdesc = pd.read_pickle('kagdesc.pkl')

kd = kdesc[['Competition Id','Overview']]

kd.to_csv('kaggdescr.csv')

doc_complete = kd[['Overview']]


stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete['Overview']]



##### try 1#####

from sklearn.feature_extraction.text import CountVectorizer

TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

vec = CountVectorizer(token_pattern = TOKENS_ALPHANUMERIC, ngram_range = (1,2))
doc_ngram = vec.fit_transform(doc_complete['Overview'])


##### try 2###

import nltk

t1 = nltk.word_tokenize(doc_complete)

########## try 3 #############

import gensim

### create a set of stopwords##
stoplist = set('a for the and to in'.split(' '))

dtm = [dictionary.doc2bow(doc) for doc in doc_clean]





########## part 2 creating dtm########

import gensim
from gensim import corpora

#creating term dictionary of the corpus, where every unique term is assigned to an index

dictionary = corpora.Dictionary(doc_clean)

#converting list of documents(corpus) into Document Term Matrix using dictionary prepared above

dtm = [dictionary.doc2bow(doc) for doc in doc_clean]


####### part 3 LDA ############3


#creating the object for LDA model using gensim library

Lda = gensim.models.ldamodel.LdaModel

# running and training LDA model on the document term matrix

ldamodel = Lda(dtm, num_topics = 10, id2word = dictionary, passes = 50)


##### results #####

print(ldamodel.print_topics(num_topics = 10, num_words = 15))







##################################################
################ archive do not touch#############
##################################################


### import dataset###

kdesc= pd.read_excel('kaggle_desc.xlsx')
kdesc.head()

kdesc.to_pickle('kagdesc.pkl')
kdesc = pd.read_pickle('kagdesc.pkl')

kd = kdesc[['Competition Id','Overview']]

kd.to_csv('kaggdescr.csv')

doc_complete = kd[['Overview']]


stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete['Overview']]


import gensim
from gensim import corpora

#creating term dictionary of the corpus, where every unique term is assigned to an index

dictionary = corpora.Dictionary(doc_clean)

#converting list of documents(corpus) into Document Term Matrix using dictionary prepared above

dtm = [dictionary.doc2bow(doc) for doc in doc_clean]


####### part 3 LDA ############3


#creating the object for LDA model using gensim library

Lda = gensim.models.ldamodel.LdaModel

# running and training LDA model on the document term matrix

ldamodel = Lda(dtm, num_topics = 10, id2word = dictionary, passes = 50)


##### results #####

print(ldamodel.print_topics(num_topics = 10, num_words = 15))

